In [ ]:
# ─────────────────────────────────────────────────────────────
# 1. Setup (GitHub Actions ready)
#    - NO pip installs here (do that in workflow)
#    - NO Colab uploads
#    - Secrets come from environment variables (GitHub Secrets)
#    - Writes: client_secrets.json, token.json
#    - Inits: openai_client
#    - Defines: BASE_OUT_DIR, create_new_run_dir()
# ─────────────────────────────────────────────────────────────

import os
import json
import pathlib
from dataclasses import dataclass
from typing import Optional
from datetime import datetime, timezone

from openai import OpenAI

# ─────────────────────────────────────────────────────────────
# A) Env helpers + config
# ─────────────────────────────────────────────────────────────

def _get_env_required(name: str) -> str:
    value = os.environ.get(name)
    if not value:
        raise ValueError(
            f"Missing required environment variable: {name}\n"
            "Set it in GitHub Actions using repository secrets, e.g.\n"
            "  env:\n"
            f"    {name}: ${{{{ secrets.{name} }}}}\n"
        )
    return value

def _get_env_optional(name: str, default: Optional[str] = None) -> Optional[str]:
    return os.environ.get(name, default)

@dataclass
class Config:
    # Core APIs
    openai_api_key: str
    suno_api_key: str
    suno_base_url: str

    # YouTube auth (stored as JSON strings in GitHub Secrets)
    youtube_client_secrets_json: str
    youtube_token_json: str

    # YouTube options
    youtube_scope: str = "https://www.googleapis.com/auth/youtube.upload"
    youtube_privacy: str = "public"

def load_config() -> Config:
    return Config(
        openai_api_key=_get_env_required("OPENAI_API_KEY"),
        suno_api_key=_get_env_required("SUNO_API_KEY"),
        suno_base_url=_get_env_optional("SUNO_BASE_URL", "https://api.sunoapi.org/api/v1").rstrip("/"),
        youtube_client_secrets_json=_get_env_required("YOUTUBE_CLIENT_SECRETS_JSON"),
        youtube_token_json=_get_env_required("YOUTUBE_TOKEN_JSON"),
        youtube_privacy=_get_env_optional("YOUTUBE_PRIVACY", "public"),
    )

# ─────────────────────────────────────────────────────────────
# B) Write YouTube JSON secrets to local files
# ─────────────────────────────────────────────────────────────

def write_youtube_secret_files(cfg: Config) -> None:
    client_path = pathlib.Path("client_secrets.json")
    token_path = pathlib.Path("token.json")

    # Validate JSON
    try:
        json.loads(cfg.youtube_client_secrets_json)
    except json.JSONDecodeError as e:
        raise ValueError("YOUTUBE_CLIENT_SECRETS_JSON is not valid JSON") from e

    try:
        token_data = json.loads(cfg.youtube_token_json)
    except json.JSONDecodeError as e:
        raise ValueError("YOUTUBE_TOKEN_JSON is not valid JSON") from e

    # Light sanity-check for token structure
    required_token_keys = {"token", "refresh_token", "scopes"}
    if not required_token_keys.issubset(token_data.keys()):
        missing = required_token_keys - set(token_data.keys())
        raise ValueError(f"YOUTUBE_TOKEN_JSON is missing required fields: {missing}")

    client_path.write_text(cfg.youtube_client_secrets_json, encoding="utf-8")
    token_path.write_text(cfg.youtube_token_json, encoding="utf-8")

    print(f"✅ Wrote YouTube secrets to {client_path} and {token_path}")

# ─────────────────────────────────────────────────────────────
# C) Output scaffolding (Actions-safe)
# ─────────────────────────────────────────────────────────────

BASE_OUT_DIR = pathlib.Path("outputs")
BASE_OUT_DIR.mkdir(parents=True, exist_ok=True)

def create_new_run_dir(base_dir: pathlib.Path) -> pathlib.Path:
    """
    Creates a unique run directory under outputs/, e.g.:
      outputs/20251231_235959_123456/
        logs/
        audio/
        images/
        video/
    """
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S_%f")
    run_dir = base_dir / ts
    (run_dir / "logs").mkdir(parents=True, exist_ok=True)
    (run_dir / "audio").mkdir(parents=True, exist_ok=True)
    (run_dir / "images").mkdir(parents=True, exist_ok=True)
    (run_dir / "video").mkdir(parents=True, exist_ok=True)
    return run_dir

# ─────────────────────────────────────────────────────────────
# D) Initialize config + OpenAI client
# ─────────────────────────────────────────────────────────────

config = load_config()

# Ensure OpenAI key is available via env as well
os.environ["OPENAI_API_KEY"] = config.openai_api_key

openai_client = OpenAI(api_key=config.openai_api_key)

write_youtube_secret_files(config)

print("✅ Config loaded successfully.")
print("   - OPENAI_API_KEY: present")
print("   - SUNO_API_KEY: present")
print(f"   - SUNO_BASE_URL: {config.suno_base_url}")
print("   - YOUTUBE_CLIENT_SECRETS_JSON: present")
print("   - YOUTUBE_TOKEN_JSON: present")
print(f"   - Default YouTube privacy: {config.youtube_privacy}")
print(f"   - BASE_OUT_DIR: {BASE_OUT_DIR.resolve()}")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 2. Preflight checks: OpenAI, Suno, YouTube, ffmpeg (GitHub Actions ready)
# ─────────────────────────────────────────────────────────────

import subprocess
import requests
import json

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# We reuse: config, openai_client, client_secrets.json, token.json

# ── OpenAI preflight ─────────────────────────────────────────

def preflight_openai(client: OpenAI):
    """Simple call to verify OpenAI key works."""
    print("🔎 Testing OpenAI connectivity...")
    try:
        models = list(client.models.list())
        print(f"✅ OpenAI OK – {len(models)} models visible.")
    except Exception:
        print("❌ OpenAI preflight failed.")
        raise


# ── Suno preflight ───────────────────────────────────────────

def preflight_suno(cfg: Config):
    """
    Connectivity check for Suno API:
    - Confirms base URL exists
    - Confirms we can reach it (no DNS/SSL/timeout errors)
    We do NOT require HTTP 200, because some APIs return 404/405 at root.
    """
    print("🔎 Testing Suno connectivity...")

    base_url = (cfg.suno_base_url or "").rstrip("/")
    if not base_url:
        raise ValueError("SUNO_BASE_URL is empty. Provide it via env or use default in Block 1.")

    headers = {"Authorization": f"Bearer {cfg.suno_api_key}"}

    try:
        resp = requests.get(base_url, headers=headers, timeout=10)
        print(f"✅ Suno reachable – HTTP {resp.status_code} from {base_url}")
    except Exception:
        print("❌ Suno preflight failed (network/SSL/timeout).")
        raise


# ── YouTube auth & preflight ─────────────────────────────────

SCOPE = config.youtube_scope

def get_authenticated_service():
    """Create authenticated YouTube client with scope validation."""
    try:
        with open("token.json", encoding="utf-8") as f:
            token_data = json.load(f)

        # Verify required upload scope is present
        if "scopes" not in token_data or SCOPE not in token_data["scopes"]:
            raise ValueError(f"Token missing required scope: {SCOPE}")

        creds = Credentials.from_authorized_user_info(token_data, [SCOPE])

        # Refresh token if needed
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
            with open("token.json", "w", encoding="utf-8") as f:
                json.dump(json.loads(creds.to_json()), f)
            print("🔄 YouTube token refreshed.")

        return build("youtube", "v3", credentials=creds)

    except HttpError as e:
        print(f"❌ YouTube API error: {e}")
        raise
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        raise


def preflight_youtube():
    """
    Confirm that we can load and refresh YouTube credentials.
    We don't call list endpoints because token scope is youtube.upload only.
    """
    print("🔎 Testing YouTube authentication...")
    yt = get_authenticated_service()
    print("✅ YouTube credentials OK – ready for uploads.")
    return yt


# ── ffmpeg preflight ─────────────────────────────────────────

def preflight_ffmpeg():
    """Ensure ffmpeg is installed and callable."""
    print("🔎 Testing ffmpeg availability...")
    try:
        result = subprocess.run(
            ["ffmpeg", "-version"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
        )
        if result.returncode == 0:
            first_line = result.stdout.splitlines()[0] if result.stdout else "ffmpeg available"
            print(f"✅ ffmpeg OK – {first_line}")
        else:
            print("❌ ffmpeg returned a non-zero exit code.")
            print(result.stderr[:500])
            raise RuntimeError("ffmpeg not working properly.")
    except FileNotFoundError:
        print("❌ ffmpeg not found on PATH.")
        raise
    except Exception:
        print("❌ ffmpeg preflight failed.")
        raise


# ── Run all preflights ───────────────────────────────────────

print("🚀 Running preflight checks...\n")

preflight_openai(openai_client)
preflight_suno(config)
youtube_client = preflight_youtube()
preflight_ffmpeg()

print("\n✅ All preflight checks passed. Safe to proceed with the rest of the pipeline.")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 3. Concept Pack Generator (Guess the Decade / Genre Edition)
#    - GitHub Actions friendly run directory creation
#    - Writes a "latest_run.txt" pointer for easy artifact pickup
# ─────────────────────────────────────────────────────────────

import random
import json
from datetime import datetime, timezone
import pathlib

# Reuse from Block 1:
# - openai_client
# - BASE_OUT_DIR
# - create_new_run_dir

TARGET_VIDEO_DURATION_S = 20.0  # aim ~20 seconds

# OPTIONAL: GitHub Actions can pass RUN_ID to make outputs deterministic
# Example in workflow:
#   env:
#     RUN_ID: ${{ github.run_id }}-${{ github.run_attempt }}
RUN_ID = os.environ.get("RUN_ID", "").strip() or None

def create_run_dir_actions(base_dir: pathlib.Path, run_id: str | None = None) -> pathlib.Path:
    """
    If run_id is provided, use outputs/<run_id>/.
    Else fall back to timestamp-based folder from create_new_run_dir().
    """
    if run_id:
        run_dir = base_dir / str(run_id)
        (run_dir / "logs").mkdir(parents=True, exist_ok=True)
        (run_dir / "audio").mkdir(parents=True, exist_ok=True)
        (run_dir / "images").mkdir(parents=True, exist_ok=True)
        (run_dir / "video").mkdir(parents=True, exist_ok=True)
        return run_dir
    return create_new_run_dir(base_dir)

# ─────────────────────────────────────────────────────────────
# OPTIONAL MANUAL OVERRIDES (easy knobs)
# ─────────────────────────────────────────────────────────────
IMAGE_PROMPT_OVERRIDE = None
IMAGE_STYLE_OVERRIDE = None
FORCE_MACRO_GENRE = None
FORCE_DECADE = None

DECADES = [
    ("1960s", "bright, punchy, tape saturation, lively band energy"),
    ("1970s", "analog warmth, funk/soul groove, roomy drums"),
    ("1980s", "synth-heavy, gated drums, bold nostalgic shine"),
    ("1990s", "raw groove, breakbeats, early digital + sample culture"),
    ("2000s", "energetic, glossy, club-ready, big hooks"),
    ("2010s", "clean modern punch, sidechain, tight low-end"),
    ("2020s", "hyper-detailed, loud/clean, hybrid genre blends"),
]

TREND_FLAVORS = [
    "jersey club bounce (fast kicks + stutter edits)",
    "latin club mutation (percussive, carnival energy)",
    "UK underground blend (garage rhythms + dark synths)",
    "hardgroove techno revival (driving hats, rolling bass)",
    "trance-tinged uplift (euphoric lead, hands-up moment)",
    "drum & bass resurgence (liquid pads or jump-up punch)",
    "afro/tribal house swing (syncopated percussion, warm bass)",
    "baile funk injection (snappy drums, call-and-response energy)",
    "phonk-adjacent grit (distorted bass, cowbell accents, but original)",
    "scandi futurism shimmer (icy synths, dreamy uplift)",
    "accelerationist trap energy (glitchy, distorted, chaotic momentum)",
    "new-london minimal cool (dry drums, intimate vibe, sharp hooks)",
]

TREATMENTS = [
    "radio edit punch (tight intro, hook-first)",
    "sped-up club cut (slightly faster, energetic)",
    "halftime switch moment (brief drop into halftime then back)",
    "beat drop at 2 seconds (immediate payoff)",
    "call-and-response hook (vocal phrase answered by synth)",
    "chopped vocal rhythm (stuttered syllables, but original words)",
    "lo-fi texture pass (subtle vinyl/tape noise, not muddy)",
    "stadium clap build (crowd claps, big impact)",
    "minimal + huge bass (spacey top, heavy low end)",
    "cinematic riser + crash (trailer-ish micro build)",
]

HOOK_MECHANICS = [
    "hook in first 0.5s (signature motif)",
    "hook in first 1s (vocal phrase + drum hit)",
    "question-like hook (vocal asks a short question)",
    "anthem hook (chanty, crowd-ready)",
    "mystery hook (odd chord + immediate groove)",
    "earworm interval jump (simple leap melody)",
    "one-word sticky hook (repeated word, but original)",
]

LYRIC_THEMES = [
    "late-night confidence",
    "summer memory flashback",
    "main character moment",
    "glow-up / level up",
    "city lights & motion",
    "heart racing / adrenaline",
    "soft nostalgia / bittersweet",
    "daydream escape",
    "victory lap energy",
    "mystery crush vibe",
    "boss-fight bravery (playful)",
    "afterparty chaos (clean words)",
]

GENRE_STACKS = {
    "Electronic": [
        "synthwave night drive",
        "festival progressive house",
        "melodic techno",
        "hardgroove techno",
        "deep house groove",
        "future bass drop",
        "drum and bass (liquid shimmer)",
        "drum and bass (jump-up punch)",
        "UK garage swing",
        "2-step garage",
        "breakbeat rave",
        "trance revival uplift",
        "tech house roller",
        "afro house groove",
        "bass house speed",
        "electroclash neon",
        "minimal house bounce",
        "IDM-ish glitch pop",
        "ambient club (soft kick, lush pads)",
    ],
    "Hip Hop / Rap": [
        "boom bap classic",
        "trap minimal",
        "trap melodic anthem",
        "dark drill energy",
        "jersey club rap bounce",
        "phonk-adjacent rap energy (original)",
        "lofi hip hop (warm, head-nod)",
        "rage synth rap (bright, aggressive)",
        "experimental glitch rap (chaotic, modern)",
    ],
    "Pop / Indie": [
        "indie bedroom pop",
        "dream pop haze",
        "alt pop anthem",
        "electropop sparkle",
        "synthpop nostalgia",
        "indie rock shimmer",
        "folk-pop stomp (claps + hook)",
        "pop punk flashback energy",
        "power ballad micro-chorus (short + hooky)",
    ],
    "R&B / Soul": [
        "alt R&B minimal (snap + sub)",
        "neo-soul groove (warm keys)",
        "R&B club slow-burn (clean vocal)",
        "funky modern soul (bass + brass)",
    ],
    "Global / Club": [
        "afrobeats bounce (clean + melodic)",
        "amapiano log-drum feel (original patterns)",
        "baile funk snap (fast percs)",
        "dancehall-pop swing",
        "latin pop club (percussive)",
    ],
    "Rock / Heavy": [
        "indie rock drive (tight drums)",
        "arena rock mini-anthem",
        "metalcore teaser (heavy, but short hook)",
        "punk energy (fast + simple chorus)",
    ],
    "Cinematic / Games / Anime": [
        "epic trailer build",
        "cyberpunk chase",
        "sci-fi atmosphere",
        "anime intro energy",
        "JRPG victory theme vibe",
        "dark fantasy tension",
        "retro arcade chase",
        "superhero reveal moment",
    ],
}

VIBE_ANCHORS = [
    "late night drive",
    "sunrise run",
    "summer festival crowd",
    "afterparty hallway",
    "alone with headphones",
    "study focus",
    "gym motivation",
    "city lights at night",
    "rainy window mood",
    "beach sunset glow",
    "first day of vacation",
    "boss fight energy",
    "victory lap moment",
    "nostalgic memory flash",
    "new year reset energy",
    "mystery movie montage",
]

VISUAL_STYLES = [
    "cinematic neon noir portrait, rain reflections, shallow depth of field",
    "anime key visual, dramatic rim light, clean linework, high contrast",
    "retro VHS glow, film grain, neon typography vibe (but NO text)",
    "high-fashion studio portrait, softbox lighting, glossy editorial look",
    "cyberpunk street scene, moody fog, glowing signage shapes (NO logos)",
    "sun-drenched festival scene, lens flare, warm bokeh, dreamy haze",
    "minimal silhouette composition, strong negative space, bold lighting",
    "futuristic sci-fi corridor, cinematic wide lens, cool tones",
    "fantasy tavern / heroic mood, painterly illustration, warm glow",
    "sports hype poster vibe, motion blur, high energy, dramatic lighting",
]

def generate_hook_phrase(decade: str, niche_genre: str, vibe: str, lyric_theme: str, hook_mechanic: str) -> str:
    prompt = (
        "Create ONE short hook phrase for a song.\n"
        "Rules:\n"
        "- 2 to 5 words only.\n"
        "- Simple English.\n"
        "- No artist names, no song titles, no brand names.\n"
        "- No profanity.\n"
        "- No punctuation except maybe a single '!'\n"
        "- Should feel catchy and repeatable.\n\n"
        f"Context:\n"
        f"- Decade feel: {decade}\n"
        f"- Niche genre: {niche_genre}\n"
        f"- Vibe: {vibe}\n"
        f"- Lyric theme: {lyric_theme}\n"
        f"- Hook mechanic: {hook_mechanic}\n\n"
        "Return ONLY the hook phrase line."
    )
    resp = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=20,
    )
    phrase = resp.choices[0].message.content.strip()
    phrase = phrase.replace('"', "").replace("'", "").strip()
    words = phrase.split()
    if len(words) > 5:
        phrase = " ".join(words[:5])
    return phrase

def generate_concept_pack():
    decade, decade_desc = random.choice(DECADES)
    if FORCE_DECADE:
        forced = [d for d in DECADES if d[0].lower() == str(FORCE_DECADE).lower()]
        if forced:
            decade, decade_desc = forced[0]

    macro_genre = random.choice(list(GENRE_STACKS.keys()))
    if FORCE_MACRO_GENRE and FORCE_MACRO_GENRE in GENRE_STACKS:
        macro_genre = FORCE_MACRO_GENRE

    niche_genre = random.choice(GENRE_STACKS[macro_genre])
    vibe = random.choice(VIBE_ANCHORS)

    trend_flavor = random.choice(TREND_FLAVORS)
    treatment = random.choice(TREATMENTS)
    hook_mechanic = random.choice(HOOK_MECHANICS)
    lyric_theme = random.choice(LYRIC_THEMES)
    visual_style = IMAGE_STYLE_OVERRIDE or random.choice(VISUAL_STYLES)

    duration_s = float(TARGET_VIDEO_DURATION_S)
    hook_phrase = generate_hook_phrase(decade, niche_genre, vibe, lyric_theme, hook_mechanic)

    prompt = (
        "You are designing an ORIGINAL short song for a short-form video.\n\n"
        "IMPORTANT RULES (must follow):\n"
        "- Imitate, reference, or recreate any specific real song, artist, melody thats similar, "
        "chord progression, topline, or recognizable motif.\n"
        "- Do NOT mention copyrighted franchises.\n"
        f"Concept:\n"
        f"- Decade feel: {decade} ({decade_desc})\n"
        f"- Macro genre: {macro_genre}\n"
        f"- Niche genre: {niche_genre}\n"
        f"- Vibe / scenario: {vibe}\n"
        f"- Trend flavor: {trend_flavor}\n"
        f"- Production treatment: {treatment}\n"
        f"- Hook mechanic: {hook_mechanic}\n"
        f"- Lyric theme: {lyric_theme}\n"
        f"- Target length: ~{int(duration_s)} seconds\n"
        f"- Hook phrase (use in vocals): {hook_phrase}\n\n"
        "Music requirements (retention critical):\n"
        "- STRONG hook in the FIRST SECOND.\n"
        "- Clear rhythm immediately; no long intro.\n"
        "- Include vocals/singing with trending words (clean, simple).\n"
        "- The hook phrase must appear clearly and be memorable.\n"
        "- End should feel loopable back to the start for a 20 second song.\n\n"
        "Describe the track clearly using:\n"
        "- Tempo / energy (include BPM estimate)\n"
        "- Drum pattern / groove\n"
        "- Bass style\n"
        "- Main instruments (2–5)\n"
        "- Vocal delivery (sung/chanty/airy/etc.)\n"
        "- Where the hook hits (timestamp-like, e.g. '0:00–0:02')\n"
        "- Short lyric direction (theme only)\n\n"
        "Return ONLY the description, no commentary."
    )

    resp = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=320,
    )
    music_prompt = resp.choices[0].message.content.strip()

    subtitle_cards = [
        {"start": 0.0,  "end": 5.0,  "text": "Can you guess the year of this song?"},
        {"start": 5.0,  "end": 10.0, "text": "What genre is it?"},
        {"start": 10.0, "end": 15.0, "text": "Reveal in 3... 2... 1..."},
        {"start": 15.0, "end": 17.5, "text": f"{decade} • {niche_genre}"},
        {"start": 17.5, "end": duration_s, "text": "What did this remind you of?"},
    ]

    if IMAGE_PROMPT_OVERRIDE and str(IMAGE_PROMPT_OVERRIDE).strip():
        image_prompt = str(IMAGE_PROMPT_OVERRIDE).strip()
    else:
        image_prompt = (
            f"A captivating photo-realistic woman. Vertical full cinematic illustration inspired by {decade} {niche_genre} music, "
            f"evoking a {vibe} feeling and {visual_style}. She has a coquettish expression, a playfully suggestive pose, and an alluring gaze. "
            f"The style balances implied sensuality with artistic taste. "
            f"No text, no logos. "
            f"Elegantly alluring, visually captivating, masterful tease."
        )

    return {
        "decade": decade,
        "macro_genre": macro_genre,
        "niche_genre": niche_genre,
        "vibe": vibe,
        "trend_flavor": trend_flavor,
        "treatment": treatment,
        "hook_mechanic": hook_mechanic,
        "lyric_theme": lyric_theme,
        "visual_style": visual_style,
        "hook_phrase": hook_phrase,
        "video_duration_s": duration_s,
        "music_prompt": music_prompt,
        "subtitle_cards": subtitle_cards,
        "image_prompt": image_prompt,
        "created_utc": datetime.now(timezone.utc).isoformat(),
    }

# ─────────────────────────────────────────────────────────────
# Create run + save concept pack
# ─────────────────────────────────────────────────────────────

RUN_DIR = create_run_dir_actions(BASE_OUT_DIR, RUN_ID)
concept_pack = generate_concept_pack()

concept_path = RUN_DIR / "logs" / "concept_pack.json"
concept_path.parent.mkdir(parents=True, exist_ok=True)
concept_path.write_text(json.dumps(concept_pack, indent=2, ensure_ascii=False), encoding="utf-8")

# Write a pointer file (helps Actions pick up newest run)
latest_ptr = BASE_OUT_DIR / "latest_run.txt"
latest_ptr.write_text(str(RUN_DIR), encoding="utf-8")

print("🎧 Concept Pack created:")
print(f"  Decade        : {concept_pack['decade']}")
print(f"  Macro genre   : {concept_pack['macro_genre']}")
print(f"  Niche genre   : {concept_pack['niche_genre']}")
print(f"  Vibe          : {concept_pack['vibe']}")
print(f"  Trend flavor  : {concept_pack['trend_flavor']}")
print(f"  Treatment     : {concept_pack['treatment']}")
print(f"  Hook mechanic : {concept_pack['hook_mechanic']}")
print(f"  Lyric theme   : {concept_pack['lyric_theme']}")
print(f"  Hook phrase   : {concept_pack['hook_phrase']}")
print(f"  Duration      : {concept_pack['video_duration_s']}s")

print("\n📁 Run directory:", RUN_DIR)
print("📄 Concept saved to:", concept_path)
print("📌 Latest run pointer:", latest_ptr)


In [ ]:
# ─────────────────────────────────────────────────────────────
# 5. Suno Custom Mode v2: generate ORIGINAL SONG (vocals ON) with synced HOOK PHRASE
#    - Ensures lyrics include concept_pack["hook_phrase"] exactly twice
# ─────────────────────────────────────────────────────────────

import json
import time
import pathlib
import requests

SUNO_BASE = config.suno_base_url.rstrip("/")
SUNO_MODEL = "V5"

def suno_headers():
    return {
        "Authorization": f"Bearer {config.suno_api_key}",
        "Content-Type": "application/json"
    }

def generate_micro_lyrics(hook_phrase: str, decade: str, niche_genre: str, vibe: str, duration_s: float, music_prompt: str) -> str:
    prompt = (
        f"Write ORIGINAL micro-lyrics for a ~{int(duration_s)} second short song.\n\n"
        "Hard rules:\n"
        "- Similar to real popular artist or song.\n"
        "- Keep lines very short (2–6 words).\n"
        "- Use simple English.\n\n"
        f"HOOK PHRASE (MUST be included EXACTLY twice, same spelling): {hook_phrase}\n\n"
        "Structure:\n"
        "[Hook]\n"
        "(hook phrase once)\n"
        "[Verse]\n"
        "(3–6 short lines describing vibe/era)\n"
        "[Hook]\n"
        "(hook phrase again)\n"
        "[Outro]\n"
        "(1 short line that loops well)\n\n"
        f"Target feel:\n- Decade: {decade}\n- Niche genre: {niche_genre}\n- Vibe: {vibe}\n"
        f"Intended length: ~{int(duration_s)} seconds\n\n"
        "Music direction (context only):\n"
        f"{music_prompt}\n\n"
        "Return ONLY the lyrics with section tags."
    )

    resp = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=230,
    )
    lyrics = resp.choices[0].message.content.strip()

    # Safety check: ensure hook phrase appears exactly twice; if not, do one repair pass
    count = lyrics.count(hook_phrase)
    if count != 2:
        repair_prompt = (
            "Fix these lyrics so that the HOOK PHRASE appears EXACTLY twice (no more, no less), "
            "spelled exactly the same.\n\n"
            f"HOOK PHRASE: {hook_phrase}\n\n"
            "Lyrics to fix:\n"
            f"{lyrics}\n\n"
            "Return ONLY the corrected lyrics."
        )
        resp2 = openai_client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": repair_prompt}],
            temperature=0.4,
            max_tokens=230,
        )
        lyrics = resp2.choices[0].message.content.strip()

    return lyrics

def suno_generate_song_with_lyrics(lyrics: str, title: str, style_hint: str, model: str = SUNO_MODEL) -> str:
    payload = {
        "customMode": True,
        "instrumental": False,   # vocals ON
        "model": model,
        "title": title[:80],
        "style": style_hint[:200],
        "prompt": lyrics,
        "callBackUrl": "https://example.com/callback"
    }

    print("🎤 Sending vocal song generate request to Suno...")
    r = requests.post(f"{SUNO_BASE}/generate", headers=suno_headers(), json=payload, timeout=60)
    r.raise_for_status()
    j = r.json()

    if j.get("code") != 200:
        raise RuntimeError(f"Suno generate error: {j}")

    task_id = j["data"]["taskId"]
    print("✅ Suno task created:", task_id)
    return task_id

def suno_record_info(task_id: str):
    r = requests.get(
        f"{SUNO_BASE}/generate/record-info",
        headers=suno_headers(),
        params={"taskId": task_id},
        timeout=60,
    )
    r.raise_for_status()
    return r.json()

def wait_for_suno_song(task_id: str, timeout_s: int = 420, poll_every: int = 5):
    print("⏳ Waiting for Suno to finish generation...")
    t0 = time.time()
    last = None
    while time.time() - t0 < timeout_s:
        last = suno_record_info(task_id)
        data = last.get("data", {})
        status = data.get("status")
        print(f"  - Status: {status}")
        if status in ("SUCCESS", "FIRST_SUCCESS"):
            print("✅ Suno generation completed.")
            return last
        time.sleep(poll_every)
    raise TimeoutError(f"Suno task timeout. Last payload: {last}")

def download_file(url: str, out_path: pathlib.Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with requests.get(url, stream=True, timeout=120) as r:
        r.raise_for_status()
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

def get_track_with_audio(task_id: str, initial_rec: dict, max_retries: int = 6, retry_delay: int = 5):
    rec = initial_rec
    for attempt in range(max_retries + 1):
        data = rec.get("data", {})
        response = data.get("response", {})
        suno_data = response.get("sunoData", []) or []

        tracks_with_url = [t for t in suno_data if t.get("audioUrl")]
        if tracks_with_url:
            if attempt > 0:
                print(f"✅ Audio URL available after {attempt} extra check(s).")
            return tracks_with_url[0]

        if attempt < max_retries:
            print(f"⚠️ audioUrl not ready yet (attempt {attempt+1}/{max_retries}), retrying...")
            time.sleep(retry_delay)
            rec = suno_record_info(task_id)

    raise RuntimeError(f"No track with a valid audioUrl found. Last payload: {rec}")

# Load concept pack
concept_path = RUN_DIR / "logs" / "concept_pack.json"
if not concept_path.exists():
    raise FileNotFoundError(f"concept_pack.json not found at {concept_path}. Run Block 3 first.")

concept_pack = json.loads(concept_path.read_text(encoding="utf-8"))

music_prompt = concept_pack["music_prompt"]
decade = concept_pack["decade"]
niche_genre = concept_pack["niche_genre"]
vibe = concept_pack["vibe"]
hook_phrase = concept_pack["hook_phrase"]
duration_s = float(concept_pack.get("video_duration_s", 17))

internal_title = f"{decade} {niche_genre}"
style_hint = f"{decade} feel, {niche_genre}, {vibe}, catchy vocal hook"

# Generate micro-lyrics synced to hook phrase
lyrics = generate_micro_lyrics(
    hook_phrase=hook_phrase,
    decade=decade,
    niche_genre=niche_genre,
    vibe=vibe,
    duration_s=duration_s,
    music_prompt=music_prompt
)

lyrics_path = RUN_DIR / "logs" / "micro_lyrics.txt"
lyrics_path.write_text(lyrics, encoding="utf-8")

print("📝 Micro-lyrics saved:", lyrics_path)
print("──── LYRICS PREVIEW ────")
print(lyrics)

# Run Suno
task_id = suno_generate_song_with_lyrics(
    lyrics=lyrics,
    title=internal_title,
    style_hint=style_hint,
    model=SUNO_MODEL
)

rec = wait_for_suno_song(task_id)
track = get_track_with_audio(task_id, rec)

AUDIO_URL = track["audioUrl"]
AUDIO_ID  = track["id"]
TITLE     = track.get("title", internal_title)
DURATION  = track.get("duration", None)

audio_path = RUN_DIR / "audio" / "song.mp3"
download_file(AUDIO_URL, audio_path)

song_meta = {
    "task_id": task_id,
    "audio_id": AUDIO_ID,
    "title": TITLE,
    "duration": DURATION,
    "audio_url": AUDIO_URL,
    "lyrics_path": str(lyrics_path),
    "concept_pack": {
        "decade": decade,
        "macro_genre": concept_pack.get("macro_genre"),
        "niche_genre": niche_genre,
        "vibe": vibe,
        "hook_phrase": hook_phrase,
        "video_duration_s": concept_pack.get("video_duration_s"),
    }
}

song_meta_path = RUN_DIR / "logs" / "suno_song_meta.json"
song_meta_path.write_text(json.dumps(song_meta, indent=2, ensure_ascii=False), encoding="utf-8")

print("🎧 Suno vocal song ready!")
print("  Title   :", TITLE)
print("  AudioID :", AUDIO_ID)
print("  Duration:", DURATION)
print("  Audio   :", audio_path)
print("  Meta    :", song_meta_path)


In [ ]:
# ─────────────────────────────────────────────────────────────
# 7. Write ASS subtitles from Concept Pack cards (tuned for vocals hook)
# ─────────────────────────────────────────────────────────────

import json
import math
from pathlib import Path

VIDEO_W = 1080
VIDEO_H = 1920
FPS = 30

concept_path = RUN_DIR / "logs" / "concept_pack.json"
if not concept_path.exists():
    raise FileNotFoundError(f"concept_pack.json not found at {concept_path}. Run Block 3 first.")

concept_pack = json.loads(concept_path.read_text(encoding="utf-8"))
cards = concept_pack.get("subtitle_cards", [])
video_duration_s = float(concept_pack.get("video_duration_s", 17))

if not cards:
    raise ValueError("No subtitle_cards found in concept_pack.json")

ASS_PATH = RUN_DIR / "subs.ass"

def ass_time(t: float) -> str:
    if t < 0:
        t = 0.0
    cs = int(round((t - math.floor(t)) * 100))
    sec = int(t) % 60
    m = (int(t) // 60) % 60
    h = int(t) // 3600
    if cs == 100:
        cs = 0
        sec += 1
    return f"{h}:{m:02d}:{sec:02d}.{cs:02d}"

def sanitize_ass_text(s: str) -> str:
    s = str(s).replace("\n", " ").strip()
    return s.replace("{", "").replace("}", "")

# Slightly larger font, stronger outline
ASS_HEADER = f"""[Script Info]
ScriptType: v4.00+
PlayResX: {VIDEO_W}
PlayResY: {VIDEO_H}
ScaledBorderAndShadow: yes

[V4+ Styles]
Format: Name,Fontname,Fontsize,PrimaryColour,SecondaryColour,OutlineColour,BackColour,Bold,Italic,Underline,StrikeOut,ScaleX,ScaleY,Spacing,Angle,BorderStyle,Outline,Shadow,Alignment,MarginL,MarginR,MarginV,Encoding
Style: FG,DejaVu Sans,78,&H00FFFFFF,&H00FFFFFF,&H00000000,&H32000000,-1,0,0,0,100,100,0,0,1,6,6,5,5,70,70,22,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""

def write_ass_from_cards(cards: list[dict], duration_s: float):
    lines = [ASS_HEADER]
    for c in cards:
        start = max(0.0, float(c["start"]))
        end = min(float(c["end"]), duration_s)
        if end <= start + 0.05:
            continue

        text = sanitize_ass_text(c["text"])
        start_s = ass_time(start)
        end_s = ass_time(end)

        # Center-middle with glow/shadow
        lines.append(
            f"Dialogue: 0,{start_s},{end_s},FG,,0,0,40,,"
            f"{{\\an5\\bord6\\blur2\\shad2}}{text}"
        )

    ASS_PATH.write_text("\n".join(lines), encoding="utf-8")

write_ass_from_cards(cards, video_duration_s)

print("✨ ASS subtitles written to:", ASS_PATH)
print("   Duration:", video_duration_s, "seconds")
print("\nPreview (first 25 lines):")
print("\n".join(ASS_PATH.read_text(encoding="utf-8").splitlines()[:25]))


In [ ]:
# ─────────────────────────────────────────────────────────────
# 8. Background image (LOW COST) + "beat-feel" motion render MP4
#    - Slow drift + micro pulse zoom (feels beat-synced)
#    - Subtle vignette to keep eyes on center
#    - Hard duration cap from concept_pack["video_duration_s"]
# ─────────────────────────────────────────────────────────────

import base64
import json
import subprocess
import pathlib

# Reuse:
# - RUN_DIR
# - openai_client
# - audio_path (from Block 5)
# - ASS_PATH (from Block 7)

concept_path = RUN_DIR / "logs" / "concept_pack.json"
if not concept_path.exists():
    raise FileNotFoundError(f"concept_pack.json not found at {concept_path}. Run Block 3 first.")

concept_pack = json.loads(concept_path.read_text(encoding="utf-8"))
video_duration_s = float(concept_pack.get("video_duration_s", 17))
image_prompt = concept_pack["image_prompt"]

BACKGROUND_IMG_PATH = RUN_DIR / "images" / "background.png"
FINAL_MP4 = RUN_DIR / "video" / "shorts_final.mp4"

# Match Block 7
VIDEO_W = 1080
VIDEO_H = 1920
FPS = 30

# Sanity checks
if not audio_path.exists():
    raise FileNotFoundError(f"Audio file not found at {audio_path}. Run Block 5 first.")
if not ASS_PATH.exists():
    raise FileNotFoundError(f"ASS subtitles not found at {ASS_PATH}. Run Block 7 first.")

# ─────────────────────────────────────────────────────────────
# OpenAI image generation (LOW COST)
# ─────────────────────────────────────────────────────────────
OPENAI_IMAGE_MODEL = "gpt-image-1-mini"
OPENAI_IMAGE_QUALITY = "low"
OPENAI_IMAGE_SIZE = "1024x1536"

def generate_background_image(prompt: str, out_path: pathlib.Path):
    print("🖼 Generating LOW-COST background image with OpenAI...")
    resp = openai_client.images.generate(
        model=OPENAI_IMAGE_MODEL,
        prompt=prompt,
        size=OPENAI_IMAGE_SIZE,
        quality=OPENAI_IMAGE_QUALITY,
        n=1,
    )
    b64 = resp.data[0].b64_json
    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_bytes(base64.b64decode(b64))
    print(f"✅ Background image saved to: {out_path}")

# Tight prompt to reduce rerolls; emphasize clean center space
bg_prompt = (
    f"{image_prompt} "
    "Vertical composition. No text, no logos, no watermarks. "
    "Simple background with a clear subject. "
    "Leave clean center space for subtitles; high contrast subject."
)

print("🧾 Image prompt (final):")
print(" ", bg_prompt)
generate_background_image(bg_prompt, BACKGROUND_IMG_PATH)

# ─────────────────────────────────────────────────────────────
# Render MP4 with "beat-feel" motion + vignette + subtitles
# ─────────────────────────────────────────────────────────────
# Motion design:
# - slow drift: a gentle oscillation (cinematic)
# - micro pulse: tiny, faster oscillation (feels beat-synced)
# Both happen inside zoompan without needing beat detection.

BASE_W = int(VIDEO_W * 1.22)
BASE_H = int(VIDEO_H * 1.22)

# Zoom = baseline + slow drift + micro pulse
# slow: sin(on/120) ~ 4 seconds per cycle @30fps (120 frames)
# pulse: sin(on/12)  ~ 0.4 seconds per cycle @30fps (12 frames)
zoom_expr = "1.035 + 0.025*sin(on/120) + 0.010*sin(on/12)"

x_expr = "iw/2 - iw/zoom/2"
y_expr = "ih/2 - ih/zoom/2"

# Vignette: mild darkening toward edges, keeps focus on subtitles center
# (FFmpeg vignette default is subtle; tune angle/strength lightly)
# Then overlay ASS subtitles.
filter_complex = (
    f"[0:v]"
    f"scale={BASE_W}:{BASE_H},"
    f"zoompan="
        f"z='{zoom_expr}':"
        f"x='{x_expr}':"
        f"y='{y_expr}':"
        f"d=1:"
        f"s={VIDEO_W}x{VIDEO_H}:"
        f"fps={FPS},"
    f"vignette=PI/5:0.15,"
    f"subtitles='{ASS_PATH.as_posix()}'"
    f"[v]"
)

ffmpeg_cmd = f"""
ffmpeg -y -loop 1 -framerate {FPS} -i "{BACKGROUND_IMG_PATH}" -i "{audio_path}" \
  -filter_complex "{filter_complex}" \
  -map "[v]" -map 1:a \
  -t {video_duration_s:.3f} \
  -c:v libx264 -preset veryfast -pix_fmt yuv420p \
  -c:a aac -b:a 192k \
  "{FINAL_MP4}"
"""

print("\n🎬 Running ffmpeg (beat-feel motion + vignette + subtitles):")
print(ffmpeg_cmd)

proc = subprocess.run(
    ["bash", "-lc", ffmpeg_cmd],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

print("── ffmpeg output ──")
print(proc.stdout)

if proc.returncode != 0:
    raise RuntimeError(f"ffmpeg failed with exit code {proc.returncode}")

print("\n✅ Final Shorts video created!")
print("   Path:", FINAL_MP4)
print("   Duration cap:", video_duration_s, "seconds")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 9. Generate Shorts metadata (Dynamic, anti-repetition, retention-first)
# ─────────────────────────────────────────────────────────────

import json
import re
import random
import unicodedata
from datetime import datetime, timezone

# Reuse:
# - RUN_DIR
# - openai_client
# - BASE_OUT_DIR

METADATA_JSON_PATH = RUN_DIR / "logs" / "metadata.json"

concept_path = RUN_DIR / "logs" / "concept_pack.json"
if not concept_path.exists():
    raise FileNotFoundError(f"concept_pack.json not found at {concept_path}. Run Block 3 first.")
concept_pack = json.loads(concept_path.read_text(encoding="utf-8"))

song_meta_path = RUN_DIR / "logs" / "suno_song_meta.json"
song_meta = json.loads(song_meta_path.read_text(encoding="utf-8")) if song_meta_path.exists() else {}

decade = concept_pack["decade"]
macro_genre = concept_pack.get("macro_genre", "")
niche_genre = concept_pack["niche_genre"]
vibe = concept_pack["vibe"]
hook_phrase = concept_pack.get("hook_phrase", "")
duration_s = float(concept_pack.get("video_duration_s", 20))

# ─────────────────────────────────────────────────────────────
# Title history (local only, per-run)
# ─────────────────────────────────────────────────────────────

TITLE_HISTORY_PATH = BASE_OUT_DIR / "title_history.json"
TITLE_HISTORY_MAX = 500

def _load_title_history(path):
    if path.exists():
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
            if isinstance(data, list):
                return [str(x) for x in data if str(x).strip()]
        except Exception:
            pass
    return []

def _save_title_history(path, titles):
    titles = titles[-TITLE_HISTORY_MAX:]
    path.write_text(json.dumps(titles, indent=2, ensure_ascii=False), encoding="utf-8")

title_history = _load_title_history(TITLE_HISTORY_PATH)
history_lower = set([t.lower() for t in title_history])

# ─────────────────────────────────────────────────────────────
# Helpers
# ─────────────────────────────────────────────────────────────

def _clean_line(s: str) -> str:
    return re.sub(r"\s+", " ", str(s)).strip()

def _dedupe_keep_order(items):
    seen = set()
    out = []
    for x in items:
        x = _clean_line(x)
        if not x:
            continue
        xl = x.lower()
        if xl in seen:
            continue
        seen.add(xl)
        out.append(x)
    return out

def _first_keyword(text: str) -> str:
    toks = re.findall(r"[A-Za-z0-9]+", (text or "").lower())
    return toks[0] if toks else ""

def _ascii_normalize(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    return s.encode("ascii", "ignore").decode("ascii")

def _clean_tag_safe(s: str) -> str:
    s = _ascii_normalize(s)
    s = s.replace("_", " ").replace("/", " ").replace("\\", " ").replace("&", " and ")
    s = re.sub(r"[^A-Za-z0-9 \-]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s.strip("-").strip()

def _normalize_title_separators(title: str) -> str:
    t = title.replace("–", "—").replace("―", "—")
    t = re.sub(r"\s*—\s*", " — ", t)
    t = re.sub(r"(?i)\b([a-z]{4,})-([a-z]{4,})\b", r"\1 - \2", t)
    t = re.sub(r"\s*-\s*", " - ", t)
    return re.sub(r"\s+", " ", t).strip("- ").strip()

TITLE_ANGLES = [
    "decade_quiz",
    "genre_quiz",
    "vibe_relate",
    "hook_focus",
    "scenario_focus",
    "wrong_answers",
    "comment_bait",
    "mystery_tease",
]
title_angle = random.choice(TITLE_ANGLES)

niche_kw = _first_keyword(niche_genre)
vibe_kw = _first_keyword(vibe)
hook_snip = hook_phrase.split()[0] if hook_phrase else ""

def generate_dynamic_metadata(
    decade, macro_genre, niche_genre, vibe, hook_phrase, duration_s, title_angle, history_examples
):
    recent_preview = "\n".join(history_examples[:30]) if history_examples else "(none)"

    prompt = f"""
You are a YouTube Shorts growth copywriter.

We publish MANY similar "guess the decade / genre" shorts.
Titles MUST vary a lot or the channel looks spammy.

Hard constraints:
- Music is ORIGINAL.
- Do NOT mention any real artists or songs.
- NO emojis, NO hashtags in titles.
- Max ~60 characters.

Video info:
- Decade: {decade}
- Macro genre: {macro_genre}
- Niche genre: {niche_genre}
- Vibe: {vibe}
- Hook phrase: {hook_phrase}
- Length: ~{int(duration_s)}s

TITLE ANGLE: {title_angle}

Recent titles:
{recent_preview}

Return VALID JSON exactly:
{{
  "title_candidates": ["..."],
  "description_candidates": ["..."],
  "tags": ["..."],
  "pinned_comment": "..."
}}
""".strip()

    resp = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.85,
        max_tokens=900,
    )

    raw = resp.choices[0].message.content.strip()
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        m = re.search(r"\{.*\}", raw, flags=re.DOTALL)
        if not m:
            raise
        return json.loads(m.group(0))

history_list_sample = list(title_history)[-120:]
random.shuffle(history_list_sample)

data = generate_dynamic_metadata(
    decade, macro_genre, niche_genre, vibe, hook_phrase, duration_s,
    title_angle, history_list_sample[:40]
)

titles = [_normalize_title_separators(_clean_line(t)) for t in data.get("title_candidates", [])]
titles = _dedupe_keep_order(titles)

chosen_title = titles[0] if titles else "This hook feels like a different era!"

descs = _dedupe_keep_order([_clean_line(d) for d in data.get("description_candidates", [])])
chosen_desc = descs[0] if descs else "New original hook — what era is it?\n\n#shorts #youtubeshorts"

tags = _dedupe_keep_order([_clean_tag_safe(t) for t in data.get("tags", []) if t])

pinned = _clean_line(data.get("pinned_comment", "")) or \
    "What did you guess first — decade or genre?"

video_meta = {
    "title": chosen_title,
    "description": chosen_desc,
    "tags": tags,
    "pinned_comment": pinned,
    "generated_utc": datetime.now(timezone.utc).isoformat(),
    "title_angle": title_angle,
}

METADATA_JSON_PATH.write_text(json.dumps(video_meta, indent=2, ensure_ascii=False), encoding="utf-8")

print("✅ Metadata saved to:", METADATA_JSON_PATH)
print("Title:", chosen_title)
print("Pinned comment:", pinned)


In [ ]:
# ─────────────────────────────────────────────────────────────
# 10. Upload final video to YouTube Shorts (ultra-safe tags + robust fallback)
#    - Reuses get_authenticated_service() from Block 2
#    - Uses config.youtube_privacy
#    - Ensures #shorts + #youtubeshorts in description
#    - Tags: ASCII-normalized, strict charset, length/total caps
#    - Fallback ladder: full -> ultra-safe -> none
#    - Saves video URL + pinned comment suggestion for manual pinning
# ─────────────────────────────────────────────────────────────

import json
import time
import re
import unicodedata
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError

# Reuse:
# - RUN_DIR
# - config (Block 1)
# - get_authenticated_service() (Block 2)

if "get_authenticated_service" not in globals():
    raise NameError("get_authenticated_service() not found. Run Block 2 first.")

FINAL_MP4 = RUN_DIR / "video" / "shorts_final.mp4"
METADATA_JSON_PATH = RUN_DIR / "logs" / "metadata.json"

if not FINAL_MP4.exists():
    raise FileNotFoundError(f"Final video not found at {FINAL_MP4}. Run Block 8 first.")
if not METADATA_JSON_PATH.exists():
    raise FileNotFoundError(f"Metadata JSON not found at {METADATA_JSON_PATH}. Run Block 9 first.")

video_meta = json.loads(METADATA_JSON_PATH.read_text(encoding="utf-8"))

title = str(video_meta.get("title", "Guess the Decade")).strip()
description = str(video_meta.get("description", "")).strip()
tags = video_meta.get("tags", [])
pinned_comment_suggestion = str(video_meta.get("pinned_comment", "")).strip()

# Ensure #shorts + #youtubeshorts (required-ish for discovery)
desc_lower = description.lower()
if "#shorts" not in desc_lower:
    description = (description.rstrip() + "\n\n#shorts").strip()
desc_lower = description.lower()
if "#youtubeshorts" not in desc_lower:
    description = (description.rstrip() + " #youtubeshorts").strip()

# Ensure tags is list[str]
if isinstance(tags, str):
    tags = [t.strip() for t in tags.split(",") if t.strip()]
elif not isinstance(tags, list):
    tags = []

def _ascii_normalize(s: str) -> str:
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    return s.encode("ascii", "ignore").decode("ascii")

def clean_youtube_tags_ultra_safe(raw_tags, max_tags=25):
    """
    Ultra-safe tag cleaning to reduce 'invalidTags' rejects:
    - ASCII normalize (removes smart quotes/em-dash/non-ascii)
    - Allow only letters/digits/spaces/hyphen
    - Collapse spaces
    - Per-tag cap 30 chars (YouTube common limit)
    - Total tag chars cap 450 (YouTube common limit)
    - Dedupe case-insensitive
    - Limit number of tags (keeps requests stable)
    """
    cleaned = []
    seen = set()
    total_len = 0

    MAX_TAG_LEN = 30
    MAX_TOTAL_LEN = 450

    for t in raw_tags:
        t = _ascii_normalize(t)
        t = t.replace("_", " ").replace("/", " ").replace("\\", " ").replace("&", " and ")
        t = t.replace("#", " ")
        t = re.sub(r"[^A-Za-z0-9 \-]", "", t)  # strict
        t = re.sub(r"\s+", " ", t).strip().strip("-").strip()

        if not t:
            continue

        if len(t) > MAX_TAG_LEN:
            t = t[:MAX_TAG_LEN].rstrip()

        tl = t.lower()
        if tl in seen:
            continue

        # Enforce caps
        if total_len + len(t) > MAX_TOTAL_LEN:
            break

        cleaned.append(t)
        seen.add(tl)
        total_len += len(t)

        if len(cleaned) >= max_tags:
            break

    return cleaned

def upload_video_to_youtube_attempt(
    youtube,
    file_path,
    title,
    description,
    tags_attempt,
    category_id="10",   # Music
    privacy="public",
    chunk_size=1024 * 1024,
    max_retries=5,
    label="",
):
    snippet = {
        "title": title,
        "description": description,
        "categoryId": str(category_id),
    }
    if tags_attempt:
        snippet["tags"] = tags_attempt

    body = {
        "snippet": snippet,
        "status": {
            "privacyStatus": privacy,
            "selfDeclaredMadeForKids": False,
        },
    }

    media = MediaFileUpload(
        str(file_path),
        mimetype="video/mp4",
        chunksize=chunk_size,
        resumable=True,
    )

    request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media,
    )

    print(f"\n🚀 Starting YouTube upload (attempt '{label}')...")
    print("Title:", title)
    print("Privacy:", privacy)
    if tags_attempt:
        print("Tags count:", len(tags_attempt), "Total tag chars:", sum(len(t) for t in tags_attempt))
        # Debug print a few tags to spot weird chars
        print("Tags sample:", tags_attempt[:8])
    else:
        print("Tags: NONE")

    response = None
    retry = 0

    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"🟢 Upload progress: {int(status.progress() * 100)}%")

        except HttpError as e:
            # invalidTags is common; bubble it so caller can fallback
            if e.resp.status == 400 and "invalidTags" in str(e):
                print(f"❌ YouTube rejected tags on attempt '{label}' (invalidTags).")
                raise

            # Transient errors -> backoff
            if e.resp.status in [500, 502, 503, 504] and retry < max_retries:
                retry += 1
                wait_for = 2 ** retry
                print(f"⚠️ Transient YouTube error {e.resp.status}. Retry {retry}/{max_retries} in {wait_for}s...")
                time.sleep(wait_for)
                continue

            print(f"❌ YouTube upload failed on attempt '{label}': {e}")
            raise

        except Exception as e:
            if retry < max_retries:
                retry += 1
                wait_for = 2 ** retry
                print(f"⚠️ Unexpected error: {e}. Retry {retry}/{max_retries} in {wait_for}s...")
                time.sleep(wait_for)
                continue
            raise

    if "id" not in response:
        raise RuntimeError(f"Unexpected YouTube API response: {response}")

    video_id = response["id"]
    print(f"✅ Upload complete on attempt '{label}'! Video ID:", video_id)
    return video_id

def upload_video_to_youtube_with_fallback(youtube, file_path, title, description, tags, privacy="public"):
    """
    Fallback ladder:
    1) Full tags from metadata (ultra-safe cleaned)
    2) Ultra-safe minimal core tags only
    3) No tags
    """
    # Attempt 1: use metadata tags (already cleaned in Block 9, but we clean again here defensively)
    full_cleaned = clean_youtube_tags_ultra_safe(tags, max_tags=25)

    try:
        return upload_video_to_youtube_attempt(
            youtube=youtube,
            file_path=file_path,
            title=title,
            description=description,
            tags_attempt=full_cleaned,
            privacy=privacy,
            label="full-ultra-safe-tags",
        )
    except HttpError as e:
        if e.resp.status == 400 and "invalidTags" in str(e):
            print("⚠️ Falling back to ultra-safe minimal tags...")
        else:
            raise

    # Attempt 2: minimal safe tags (intentionally boring but reliable)
    minimal = clean_youtube_tags_ultra_safe([
        "guess the decade",
        "guess the era",
        "guess the genre",
        "music quiz",
        "music challenge",
        "original music",
        "shorts music",
        "youtube shorts",
    ], max_tags=12)

    try:
        return upload_video_to_youtube_attempt(
            youtube=youtube,
            file_path=file_path,
            title=title,
            description=description,
            tags_attempt=minimal,
            privacy=privacy,
            label="minimal-ultra-safe-tags",
        )
    except HttpError as e:
        if e.resp.status == 400 and "invalidTags" in str(e):
            print("⚠️ Falling back to NO tags...")
        else:
            raise

    # Attempt 3: no tags (should always upload)
    return upload_video_to_youtube_attempt(
        youtube=youtube,
        file_path=file_path,
        title=title,
        description=description,
        tags_attempt=[],
        privacy=privacy,
        label="no-tags",
    )

# ── Run upload ───────────────────────────────────────────────
youtube = get_authenticated_service()
privacy = getattr(config, "youtube_privacy", "public")

video_id = upload_video_to_youtube_with_fallback(
    youtube=youtube,
    file_path=FINAL_MP4,
    title=title,
    description=description,
    tags=tags,
    privacy=privacy,
)

video_url = f"https://youtu.be/{video_id}"
print("\n🎉 Video uploaded:")
print("   ", video_url)

# Save URL + pinned comment suggestion in run folder
(RUN_DIR / "logs" / "video_url.txt").write_text(video_url, encoding="utf-8")
if pinned_comment_suggestion:
    (RUN_DIR / "logs" / "pinned_comment.txt").write_text(pinned_comment_suggestion, encoding="utf-8")

print("\n📌 Pinned comment suggestion (saved to logs/pinned_comment.txt):")
print(pinned_comment_suggestion or "(none)")
